# Exercise 1: Hello world

1) Write an MPI program which prints the message "Hello World"

In [1]:
%%file quiz1.py
from mpi4py import MPI

COMM = MPI.COMM_WORLD
RANK = COMM.Get_rank()
SIZE = COMM.Get_size()

# Print "Hello World" from each process
print(f"Hello World" )

Writing quiz1.py


In [2]:
!mpiexec -n 4 python quiz1.py

Hello World
Hello World
Hello World
Hello World


2. Modify your program so that each process prints out both its rank and the total number of processes P that the code is running on, i.e. the size of MPI_COMM_WORLD.

In [3]:
%%file quiz2.py
from mpi4py import MPI

COMM = MPI.COMM_WORLD
RANK = COMM.Get_rank()
SIZE = COMM.Get_size()

print(f"Hello World from process {RANK} of {SIZE}".format ( RANK = RANK , SIZE = SIZE ) )

Writing quiz2.py


In [4]:
!mpiexec -n 4 python quiz2.py

Hello World from process 3 of 4
Hello World from process 2 of 4
Hello World from process 1 of 4
Hello World from process 0 of 4


3. Modify your program so that only a single controller process (e.g. rank 0) prints out a message (very useful when you run with hundreds of processes).

In [5]:
%%file quiz3.py
from mpi4py import MPI

COMM = MPI.COMM_WORLD
RANK = COMM.Get_rank()
SIZE = COMM.Get_size()

if RANK == 0:
    print(f"Hello World from process {RANK} of {SIZE}".format ( RANK = RANK , SIZE = SIZE ) )

Writing quiz3.py


In [6]:
!mpiexec -n 4 python quiz3.py

Hello World from process 0 of 4


4. What happens if you omit the final MPI procedure call in your program?

Python le met automatiquement même si on ne le met pas sinon ça ne devrait pas marcher.

# Exercise 2: Sharing Data

1) Create a program that obtains an integer input from the terminal and distributes it to all the MPI processes. Each process must display its rank and the received value. Keep reading values until a negative integer is entered.

In [9]:
%%file exo2.py

from mpi4py import MPI

COMM = MPI.COMM_WORLD
RANK = COMM.Get_rank()
SIZE = COMM.Get_size()

a = 0
while True:
    try:
        if RANK == 0:
            a = int(input("Entrez un entier: "))
            for p in range(1,SIZE):
                COMM.send(a, dest = p)
                
        else:
            a = COMM.recv(source = 0)

        if a < 0:
            break

        print(f"Process {RANK} got {a}")
        
    except ValueError:
        print("Entree invalide.")

Overwriting exo2.py


In [10]:
!mpiexec -n 4 python exo2.py

^C


# Exercise 3: Ping-Pong

In [142]:
%%file exo3.py

from mpi4py import MPI

COMM = MPI.COMM_WORLD
RANK = COMM.Get_rank()
SIZE = COMM.Get_size()

itrs = 10
a = 0

for i in range(itrs):
    if RANK == 0:
        COMM.send(a, dest=1)
        a = COMM.recv(source=1)
        print(f"Iteration {i}: I, process {RANK}, I received {a} from the process 1.")
        
    if RANK == 1:
        a = COMM.recv(source=0)
        print(f"Iteration {i}: I, process {RANK}, I received {a} from the process 0.")
        a += 1
        COMM.send(a, dest=0)


Writing exo3.py


In [ ]:
!mpiexec -n 4 python exo3.py

# Exercise 4: Sending in a ring (Broadcast by ring)

Write a program that takes data from process zero and sends it to all of the other processes by sending it in a ring. That is, process i should receive the data add the rank of the process to it then send it to process i+1, until the last process is reached. Assume that the data consists of a single integer. Process zero reads the data from the user, print the process rank and the value received.

In [135]:
%%file exo4.py

from mpi4py import MPI

COMM = MPI.COMM_WORLD
RANK = COMM.Get_rank()
SIZE = COMM.Get_size()
data = 0
if RANK == 0:
    try:
        data = int(input("Entrez un entier: "))
    except ValueError:
        print("Entree invalide.")
        exit()
        
    print(f"I, process {RANK}, I received {data} from the process {RANK}.")
        
    COMM.send(data, dest=1)
        
else:
    data = COMM.recv(source=(RANK - 1))
    print(f"I, process {RANK}, I received {data} from the process {(RANK - 1)}.")

    if RANK != SIZE - 1:
        data += RANK
        COMM.send(data, dest=(RANK + 1))    
    

Overwriting exo4.py


In [ ]:
!mpiexec -n 4 python exo4.py

# Exercise 5: Solving 1D advection equation 

Implement a parallel solver for the 1D advection equation using the function above:

In [9]:
%%file exo5.py

from mpi4py import MPI
import numpy as np
import matplotlib.pyplot as plt
import time

COMM = MPI.COMM_WORLD
RANK = COMM.Get_rank()
SIZE = COMM.Get_size()

start_time = 0
end_time = 0

def graphe(x,u, ti):
    plt.plot(x,u, '-b', label=ti)
    plt.grid()
    plt.xlabel('x')
    plt.ylabel('u')
    plt.title('Courbe')
    plt.legend()
    plt.show()

nx = 200
try:
    assert nx >= SIZE
except AssertionError:
    print("Nombre de processus superieur au nombre de points.")


CFL = 1
c = 1
L = 1000
x = np.linspace(0,L,nx)
dx = L/(nx-1)
dt = CFL*dx/abs(c)
u = np.zeros(nx)

for i in range(nx):
    if 300 < x[i] < 400:
        u[i] = 10
    else:
        u[i] = 0

if RANK==0:
    graphe(x,u,"Solution initiale")

def solve_1d_linearconv(u, un, nt, nx, dt, dx, c):

    for n in range(nt):  
        un = np.copy(u)
        
        for i in range(1, nx): 
            u[i] = un[i] - c * dt / dx * (un[i] - un[i-1])
            
        # Envoi des bords au processus voisin
        if RANK < SIZE-1:
            COMM.send(u[-1], dest=RANK+1, tag=2)
        if RANK != 0:
            u[0] = COMM.recv(source=RANK - 1, tag=2)
            
    return 0

#---------------------------
# Calcul ||
#---------------------------

start_time = time.time()

if RANK == 0:
    # Le partage du domaine aux process
    u_loc = np.array_split(u,SIZE)
    
    for i in range(SIZE-1):
        u_loc[i+1] = np.insert(u_loc[i+1], 0, u_loc[i][-1])
        
    # L'envoi des domaines à chaque process
    for process in range(SIZE):
        COMM.send(u_loc[process], dest=process, tag=0)

# Reception des domaines et calcul
u = COMM.recv(source = 0, tag = 0)
nx = len(u)
un = np.zeros(nx)
nt = 10

solve_1d_linearconv(u, un, nt, nx, dt, dx, c)
COMM.send(u[1:], dest=0, tag=1)

u_final = np.array([0])
if RANK == 0:
    for i in range(SIZE):
        un_loc = COMM.recv(source=i, tag=1)
        u_final = np.concatenate((u_final, un_loc))
        
    graphe(x,un, "Solution finale")
    
end_time = time.time()
total_time = end_time - start_time


Overwriting exo5.py


In [ ]:
!mpiexec -n 4 python exo5.py